## Understanding Seasonal Trends through NLP
- Abigail Bojorquez
- Zachary Romero

### Project Proposal: 
Analyzing Twitter data visualizes trends in common cold/flu symptoms vs pneumonia/bronchitis symptoms to educate health professionals.


In [1]:
## Standard Imports
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import spacy 
import csv
import tweepy as tweepy
from spacy.lang.en.stop_words import STOP_WORDS
from geopy.geocoders import Nominatim
from textblob import TextBlob
## Importing the OS and JSON Modules
import os
import json

geolocator = Nominatim(user_agent="UCI_Tweet_Illness_App")
nlp = spacy.load('en_core_web_sm')

## Importing API Credentials

In [2]:
with open('/Users/abigailbojorquez/.secret/twitter_api.json') as f:
    login = json.load(f)
    
    login.keys()

In [3]:
auth = tweepy.OAuthHandler(login['consumer_key'], login['consumer_secret'])
auth.set_access_token(login['access_token'], login['access_secret'])
 
twitter_api = tweepy.API(auth)

In [4]:
# Quick Test Query
search_words = 'cold OR flu OR pneumonia OR bronchitis'

results = tweepy.Cursor(twitter_api.search_tweets, 
                        q=search_words, 
                        lang='en', 
                        since_id='2023-02-20').items(5)
for tweets in results:
    print(tweets.text)

RT @gyumedia: jeonghan suddenly telling mingyu off because he wanted to be with him 😅

🐶 it's seriously so cold there
😇 you're cold because…
RT @osullivanauthor: Ted Cruz called Joe Biden spineless.

That's comical.  

Joe Biden went to Ukraine during a war.
Ted Cruz went to Mexi…
RT @JackedAecus: 1) Cold Showers Build Strong Will Power

It takes a tremendous amount of willpower to take a cold shower on a cold day.
It's really cold today so he gotta snuggle up 🥺 https://t.co/GYP2qu4WKN
RT @translatingTXT: 💛 there’s even boys planet going on right now so can i go?
🧸 ha.ha.ha you’re so funny
🧸 that was really funny, that was…


In [5]:
# this will tell us how many
limits = twitter_api.rate_limit_status()
print('Calls Left: ')
print(limits['resources']['search']['/search/tweets']['remaining'])
reset_time = limits['resources']['search']['/search/tweets']['reset']
dt = datetime.fromtimestamp(reset_time)
print('\nTime Until Reset: ')
print(dt)

Calls Left: 
179

Time Until Reset: 
2023-02-23 18:05:33


In [6]:
# Quick Test Query

search_words = 'cold OR flu OR pneumonia OR bronchitis since:2022-01-01 -filter:retweets'
location = "39.8,-95.583068847656,2500km"  # Geographical center of the US with 2500km radius
language = "en"
result_type = "recent"

tweets = tweepy.Cursor(twitter_api.search_tweets, 
                        q=search_words, 
                        geocode=location,
                        lang=language).items(1000)

### Filters Tweets and Import into Data Frame

In [7]:
filtered_tweets = []

for tweet in tweets:
    if tweet.place is not None:
        filtered_tweets.append({
            'text': tweet.text,
            'location': tweet.place.full_name,
            'date': tweet.created_at.date()
        })
        
tweets_with_loc = pd.DataFrame(filtered_tweets)
tweets_with_loc.to_csv('test_tweets.csv', index=False)

## Creating Data Frame of Test_tweets

In [8]:
df = pd.read_csv('test_tweets.csv')
df.head()

,text,location,date
0,"THEFT - COLD at 6800 N TRUMBULL AVE, PORT [Por...","Portland, OR",2023-02-24
1,@onterrorstreet Flash back to Carl shooting hi...,"Rideau Lakes, Ontario",2023-02-24
2,@MarkyDee3 Bro I seen em shooting jump shots a...,"Kalamazoo, MI",2023-02-24
3,"ASSAULT - COLD at 11400 SE DIVISION ST, PORT [...","Portland, OR",2023-02-24
4,@habby44 @ArponBasu Sounds like he has the Phi...,"Fort Saskatchewan, Alberta",2023-02-24
